# <span style="color:mediumvioletred"> **Data Wrangling**</span>
### **Proyecto de Ciencia de Datos**
- Aissa Berenice Gonzalez Fosado 752224
- Clara Paola Aguilar Casillas
- Daniela de la Torre Gallo


<img src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal-Gris.png"  width="300">

##  **Data Wrangling**
- Reglas aplicadas por tema: nulos, duplicados, outliers, casting/encoding, reglas de negocio.
- Mini-evidencias **antes/después** (tabla/figura).
- Estado del dataset limpio (ruta en `data/processed/` y esquema final).

In [30]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset
df = pd.read_csv("../data/raw/apartments.csv", sep=";", encoding='cp1252')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   category       10000 non-null  object 
 2   title          10000 non-null  object 
 3   body           10000 non-null  object 
 4   amenities      6451 non-null   object 
 5   bathrooms      9966 non-null   float64
 6   bedrooms       9993 non-null   float64
 7   currency       10000 non-null  object 
 8   fee            10000 non-null  object 
 9   has_photo      10000 non-null  object 
 10  pets_allowed   5837 non-null   object 
 11  price          10000 non-null  int64  
 12  price_display  10000 non-null  object 
 13  price_type     10000 non-null  object 
 14  square_feet    10000 non-null  int64  
 15  address        6673 non-null   object 
 16  cityname       9923 non-null   object 
 17  state          9923 non-null   object 
 18  latitud

In [33]:
df.head(2)

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,pets_allowed,price,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668626895,housing/rent/apartment,"Studio apartment 2nd St NE, Uhland Terrace NE,...","This unit is located at second St NE, Uhland T...",NaN,NaN,0.0,USD,No,Thumbnail,NaN,790,$790,Monthly,101,NaN,Washington,DC,38.9057,-76.9861,RentLingo,1577359415
1,5664597177,housing/rent/apartment,Studio apartment 814 Schutte Road,"This unit is located at 814 Schutte Road, Evan...",NaN,NaN,1.0,USD,No,Thumbnail,NaN,425,$425,Monthly,106,814 Schutte Rd,Evansville,IN,37.9680,-87.6621,RentLingo,1577017063


We start by inspecting the structure of the dataset — number of rows, columns, and data types. This helps us identify numerical, categorical, and text fields.
python
Copiar código


Comenzamos inspeccionando la estructura del dataset, el número de filas, columnas y tipo de datos que vimos en el EDA, esto nos ayuda a identificar el tipo de datos que estaremos manejando.

La primer modificación que realizamos fué de

### 🧠 Data Types & Missing Values

We will now check which columns have missing data and how much. Handling missing data correctly is crucial to avoid bias or errors during modeling.


In [ ]:
# Check for missing values
missing = df.isnull().sum().sort_values(ascending=False)
missing[missing > 0]

In [3]:
# Visualize missing data
plt.figure(figsize=(10, 5))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Values Heatmap")
plt.show()

pets_allowed    4163
amenities       3549
address         3327
state             77
cityname          77
bathrooms         34
longitude         10
latitude          10
bedrooms           7
dtype: int64

### 🔍 Summary Statistics

This step helps us understand the spread, distribution, and potential outliers in numerical features.

In [ ]:
# Summary statistics
df.describe().T

### ⚙️ Data Cleaning

We will clean duplicates, handle missing values, and standardize column names.

In [ ]:
# Copy dataframe to keep original safe
df_clean = df.copy()

# --- Fill missing categorical/text columns ---
df_clean['pets_allowed'].fillna('unknown', inplace=True)
df_clean['amenities'].fillna('Not specified', inplace=True)
df_clean['address'].fillna('Not provided', inplace=True)

# Fill with mode (most common value)
for col in ['state', 'cityname']:
    mode_value = df_clean[col].mode()[0]
    df_clean[col].fillna(mode_value, inplace=True)

# --- Fill missing numeric columns ---
for col in ['bathrooms', 'bedrooms', 'latitude', 'longitude']:
    median_value = df_clean[col].median()
    df_clean[col].fillna(median_value, inplace=True)


### ✅ Validation After Cleaning

We’ll now verify that no missing values remain in the dataset.


In [ ]:
# Check for remaining missing values
df_clean.isnull().sum().sort_values(ascending=False)


## 🚨 Outlier Detection and Treatment

Outliers can distort statistical analysis and model performance — especially in features like `price`, `bathrooms`, and `bedrooms`.
We’ll detect them using the **Interquartile Range (IQR)** method and decide whether to remove or cap them.


In [ ]:
# Select numeric columns for outlier detection
numeric_cols = ['price', 'bedrooms', 'bathrooms', 'latitude', 'longitude']

# Visual inspection with boxplots
plt.figure(figsize=(12, 6))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x=df_clean[col], color='skyblue')
    plt.title(f'{col} - Boxplot')
plt.tight_layout()
plt.show()


### 🔍 Identifying Outliers with IQR

We’ll calculate Q1 (25th percentile) and Q3 (75th percentile) for each numeric feature, then define outliers as values beyond **1.5 × IQR**.


In [ ]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    return outliers, lower, upper

# Example check for price
outliers_price, low_p, high_p = detect_outliers_iqr(df_clean, 'price')
print(f"Outliers in 'price': {len(outliers_price)} | Lower bound: {low_p:.2f} | Upper bound: {high_p:.2f}")


### ✂️ Handling Outliers

There are several strategies:
- **Remove:** For extreme and unrealistic values.
- **Cap:** Replace with upper/lower limits to preserve distribution.

We’ll apply **capping**, as real estate prices often contain valid high-end values.


In [ ]:
# Apply IQR-based capping for numeric columns
for col in ['price', 'bathrooms', 'bedrooms']:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df_clean[col] = np.where(df_clean[col] < lower, lower, df_clean[col])
    df_clean[col] = np.where(df_clean[col] > upper, upper, df_clean[col])


### ✅ Validation After Outlier Treatment

We’ll check again using boxplots to ensure extreme values were capped correctly.


In [ ]:
plt.figure(figsize=(12, 6))
for i, col in enumerate(['price', 'bathrooms', 'bedrooms'], 1):
    plt.subplot(1, 3, i)
    sns.boxplot(x=df_clean[col], color='lightgreen')
    plt.title(f'{col} (After Capping)')
plt.tight_layout()
plt.show()


Our dataset is now:
- Free of missing values ✅
- Cleaned of extreme outliers ✅
- Ready for feature engineering and modeling 🚀


Our dataset is now:
- Free of missing values ✅
- Cleaned of extreme outliers ✅
- Ready for feature engineering and modeling 🚀
